# Loading packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import gc
import keras_cv

from os import path
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model, Input, regularizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9048)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

1 Physical GPU, 1 Logical GPUs


In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 224
BATCH_SIZE = 512
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Loading test dataset

In [5]:
dataset_test = image_dataset_from_directory(
    directory=path.join(os.getcwd(), '..', 'cifar-10', 'test'),
    labels=None,
    label_mode=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    batch_size=512
)
dataset_test = dataset_test.map(lambda x : preprocess_input(x)).prefetch(AUTOTUNE)

Found 300000 files belonging to 1 classes.


In [6]:
def get_model():
    resnet = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights="imagenet", include_top=False)

    for i, layer in enumerate(resnet.layers):
        if not layer.name.startswith('conv5'):
            layer.trainable = False

    x = layers.GlobalAveragePooling2D()(resnet.output)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(2048, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(N_CLASS, activation="softmax")(x)
    model = Model(resnet.input, output)
    return model

model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [7]:
for i in range(10):
    model = get_model()
    model.load_weights(f"weights/weights_{i}.h5")
    eval_results = model.predict(dataset_test)
    np.save(f"results/result_{i}", eval_results)
    gc.collect()

586/586 [==============================] - 228s 384ms/step
